# AutoKeras

* https://autokeras.com/

## Imports and Global Settings

In [1]:
import sys
import datetime
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    mean_absolute_error,
    precision_score,
    r2_score,
)

sys.path.append('../')
from passkeys import RDS_ENDPOINT, RDS_PASSWORD

# Pandas Settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.max_info_columns = 200
pd.options.display.precision = 5

2023-03-02 01:00:56.622914: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 01:00:56.762438: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-02 01:00:56.762455: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-02 01:00:57.492349: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Database Connection

In [2]:
username = 'postgres'
password = RDS_PASSWORD
endpoint = RDS_ENDPOINT
database = 'nba_betting'
port = '5432'

connection = create_engine(f'postgresql+psycopg2://{username}:{password}@{endpoint}/{database}').connect()

### Datasets

In [3]:
df = pd.read_sql_table('model_training_data', connection)

### Restrict to previous years games only

In [4]:
df = df[df['league_year_end'] != 23]

In [5]:
df.sort_values('game_id', ascending=False).head()

,game_id,CLS_TARGET_home_margin_GT_home_spread,REG_TARGET_actual_home_margin,home_team_num,away_team_num,league_year_end,fd_line_home,dk_line_home,covers_consensus_home,home_spread,gp,win,loss,w_pct,mins,pts,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,blka,pf,pfd,p_m,gp_rank,gp_zscore,win_rank,win_zscore,loss_rank,loss_zscore,w_pct_rank,w_pct_zscore,mins_rank,mins_zscore,pts_rank,pts_zscore,fgm_rank,fgm_zscore,fga_rank,fga_zscore,fg_pct_rank,fg_pct_zscore,fg3m_rank,fg3m_zscore,fg3a_rank,fg3a_zscore,fg3_pct_rank,fg3_pct_zscore,ftm_rank,ftm_zscore,fta_rank,fta_zscore,ft_pct_rank,ft_pct_zscore,oreb_rank,oreb_zscore,dreb_rank,dreb_zscore,reb_rank,reb_zscore,ast_rank,ast_zscore,tov_rank,tov_zscore,stl_rank,stl_zscore,blk_rank,blk_zscore,blka_rank,blka_zscore,pf_rank,pf_zscore,pfd_rank,pfd_zscore,p_m_rank,p_m_zscore,gp_opp,win_opp,loss_opp,w_pct_opp,mins_opp,pts_opp,fgm_opp,fga_opp,fg_pct_opp,fg3m_opp,fg3a_opp,fg3_pct_opp,ftm_opp,fta_opp,ft_pct_opp,oreb_opp,dreb_opp,reb_opp,ast_opp,tov_opp,stl_opp,blk_opp,blka_opp,pf_opp,pfd_opp,p_m_opp,gp_rank_opp,gp_zscore_opp,win_rank_opp,win_zscore_opp,loss_rank_opp,loss_zscore_opp,w_pct_rank_opp,w_pct_zscore_opp,mins_rank_opp,mins_zscore_opp,pts_rank_opp,pts_zscore_opp,fgm_rank_opp,fgm_zscore_opp,fga_rank_opp,fga_zscore_opp,fg_pct_rank_opp,fg_pct_zscore_opp,fg3m_rank_opp,fg3m_zscore_opp,fg3a_rank_opp,fg3a_zscore_opp,fg3_pct_rank_opp,fg3_pct_zscore_opp,ftm_rank_opp,ftm_zscore_opp,fta_rank_opp,fta_zscore_opp,ft_pct_rank_opp,ft_pct_zscore_opp,oreb_rank_opp,oreb_zscore_opp,dreb_rank_opp,dreb_zscore_opp,reb_rank_opp,reb_zscore_opp,ast_rank_opp,ast_zscore_opp,tov_rank_opp,tov_zscore_opp,stl_rank_opp,stl_zscore_opp,blk_rank_opp,blk_zscore_opp,blka_rank_opp,blka_zscore_opp,pf_rank_opp,pf_zscore_opp,pfd_rank_opp,pfd_zscore_opp,p_m_rank_opp,p_m_zscore_opp,offrtg,defrtg,netrtg,ast_pct,ast_v_tov,ast_ratio,oreb_pct,dreb_pct,reb_pct,tov_pct,efg_pct,ts_pct,pace,pie,poss,offrtg_rank,offrtg_zscore,defrtg_rank,defrtg_zscore,netrtg_rank,netrtg_zscore,ast_pct_rank,ast_pct_zscore,ast_v_tov_rank,ast_v_tov_zscore,ast_ratio_rank,ast_ratio_zscore,oreb_pct_rank,oreb_pct_zscore,dreb_pct_rank,dreb_pct_zscore,reb_pct_rank,reb_pct_zscore,tov_pct_rank,tov_pct_zscore,efg_pct_rank,efg_pct_zscore,ts_pct_rank,ts_pct_zscore,pace_rank,pace_zscore,pie_rank,pie_zscore,poss_rank,poss_zscore,offrtg_opp,defrtg_opp,netrtg_opp,ast_pct_opp,ast_v_tov_opp,ast_ratio_opp,oreb_pct_opp,dreb_pct_opp,reb_pct_opp,tov_pct_opp,efg_pct_opp,ts_pct_opp,pace_opp,pie_opp,poss_opp,offrtg_rank_opp,offrtg_zscore_opp,defrtg_rank_opp,defrtg_zscore_opp,netrtg_rank_opp,netrtg_zscore_opp,ast_pct_rank_opp,ast_pct_zscore_opp,ast_v_tov_rank_opp,ast_v_tov_zscore_opp,ast_ratio_rank_opp,ast_ratio_zscore_opp,oreb_pct_rank_opp,oreb_pct_zscore_opp,dreb_pct_rank_opp,dreb_pct_zscore_opp,reb_pct_rank_opp,reb_pct_zscore_opp,tov_pct_rank_opp,tov_pct_zscore_opp,efg_pct_rank_opp,efg_pct_zscore_opp,ts_pct_rank_opp,ts_pct_zscore_opp,...,fga_c3_opp,fg_pct_c3_opp,fgm_atb3_opp,fga_atb3_opp,fg_pct_atb3_opp,fgm_ra_rank_opp,fgm_ra_zscore_opp,fga_ra_rank_opp,fga_ra_zscore_opp,fg_pct_ra_rank_opp,fg_pct_ra_zscore_opp,fgm_paint_rank_opp,fgm_paint_zscore_opp,fga_paint_rank_opp,fga_paint_zscore_opp,fg_pct_paint_rank_opp,fg_pct_paint_zscore_opp,fgm_mr_rank_opp,fgm_mr_zscore_opp,fga_mr_rank_opp,fga_mr_zscore_opp,fg_pct_mr_rank_opp,fg_pct_mr_zscore_opp,fgm_lc3_rank_opp,fgm_lc3_zscore_opp,fga_lc3_rank_opp,fga_lc3_zscore_opp,fg_pct_lc3_rank_opp,fg_pct_lc3_zscore_opp,fgm_rc3_rank_opp,fgm_rc3_zscore_opp,fga_rc3_rank_opp,fga_rc3_zscore_opp,fg_pct_rc3_rank_opp,fg_pct_rc3_zscore_opp,fgm_c3_rank_opp,fgm_c3_zscore_opp,fga_c3_rank_opp,fga_c3_zscore_opp,fg_pct_c3_rank_opp,fg_pct_c3_zscore_opp,fgm_atb3_rank_opp,fgm_atb3_zscore_opp,fga_atb3_rank_opp,fga_atb3_zscore_opp,fg_pct_atb3_rank_opp,fg_pct_atb3_zscore_opp,opp_fgm_ra,opp_fga_ra,opp_fg_pct_ra,opp_fgm_paint,opp_fga_paint,opp_fg_pct_paint,opp_fgm_mr,opp_fga_mr,opp_fg_pct_mr,opp_fgm_lc3,opp_fga_lc3,opp_fg_pct_lc3,opp_fgm_

## Basic Data Overview

In [6]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10195 entries, 0 to 10208
Data columns (total 921 columns):
 #    Column                                 Non-Null Count  Dtype  
---   ------                                 --------------  -----  
 0    game_id                                10195 non-null  object 
 1    CLS_TARGET_home_margin_GT_home_spread  10195 non-null  bool   
 2    REG_TARGET_actual_home_margin          10195 non-null  float64
 3    home_team_num                          10195 non-null  int64  
 4    away_team_num                          10195 non-null  int64  
 5    league_year_end                        10195 non-null  int64  
 6    fd_line_home                           0 non-null      object 
 7    dk_line_home                           0 non-null      object 
 8    covers_consensus_home                  0 non-null      object 
 9    home_spread                            10195 non-null  float64
 10   gp                                     9396 non-null   o

In [7]:
df.head()

,game_id,CLS_TARGET_home_margin_GT_home_spread,REG_TARGET_actual_home_margin,home_team_num,away_team_num,league_year_end,fd_line_home,dk_line_home,covers_consensus_home,home_spread,gp,win,loss,w_pct,mins,pts,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,blka,pf,pfd,p_m,gp_rank,gp_zscore,win_rank,win_zscore,loss_rank,loss_zscore,w_pct_rank,w_pct_zscore,mins_rank,mins_zscore,pts_rank,pts_zscore,fgm_rank,fgm_zscore,fga_rank,fga_zscore,fg_pct_rank,fg_pct_zscore,fg3m_rank,fg3m_zscore,fg3a_rank,fg3a_zscore,fg3_pct_rank,fg3_pct_zscore,ftm_rank,ftm_zscore,fta_rank,fta_zscore,ft_pct_rank,ft_pct_zscore,oreb_rank,oreb_zscore,dreb_rank,dreb_zscore,reb_rank,reb_zscore,ast_rank,ast_zscore,tov_rank,tov_zscore,stl_rank,stl_zscore,blk_rank,blk_zscore,blka_rank,blka_zscore,pf_rank,pf_zscore,pfd_rank,pfd_zscore,p_m_rank,p_m_zscore,gp_opp,win_opp,loss_opp,w_pct_opp,mins_opp,pts_opp,fgm_opp,fga_opp,fg_pct_opp,fg3m_opp,fg3a_opp,fg3_pct_opp,ftm_opp,fta_opp,ft_pct_opp,oreb_opp,dreb_opp,reb_opp,ast_opp,tov_opp,stl_opp,blk_opp,blka_opp,pf_opp,pfd_opp,p_m_opp,gp_rank_opp,gp_zscore_opp,win_rank_opp,win_zscore_opp,loss_rank_opp,loss_zscore_opp,w_pct_rank_opp,w_pct_zscore_opp,mins_rank_opp,mins_zscore_opp,pts_rank_opp,pts_zscore_opp,fgm_rank_opp,fgm_zscore_opp,fga_rank_opp,fga_zscore_opp,fg_pct_rank_opp,fg_pct_zscore_opp,fg3m_rank_opp,fg3m_zscore_opp,fg3a_rank_opp,fg3a_zscore_opp,fg3_pct_rank_opp,fg3_pct_zscore_opp,ftm_rank_opp,ftm_zscore_opp,fta_rank_opp,fta_zscore_opp,ft_pct_rank_opp,ft_pct_zscore_opp,oreb_rank_opp,oreb_zscore_opp,dreb_rank_opp,dreb_zscore_opp,reb_rank_opp,reb_zscore_opp,ast_rank_opp,ast_zscore_opp,tov_rank_opp,tov_zscore_opp,stl_rank_opp,stl_zscore_opp,blk_rank_opp,blk_zscore_opp,blka_rank_opp,blka_zscore_opp,pf_rank_opp,pf_zscore_opp,pfd_rank_opp,pfd_zscore_opp,p_m_rank_opp,p_m_zscore_opp,offrtg,defrtg,netrtg,ast_pct,ast_v_tov,ast_ratio,oreb_pct,dreb_pct,reb_pct,tov_pct,efg_pct,ts_pct,pace,pie,poss,offrtg_rank,offrtg_zscore,defrtg_rank,defrtg_zscore,netrtg_rank,netrtg_zscore,ast_pct_rank,ast_pct_zscore,ast_v_tov_rank,ast_v_tov_zscore,ast_ratio_rank,ast_ratio_zscore,oreb_pct_rank,oreb_pct_zscore,dreb_pct_rank,dreb_pct_zscore,reb_pct_rank,reb_pct_zscore,tov_pct_rank,tov_pct_zscore,efg_pct_rank,efg_pct_zscore,ts_pct_rank,ts_pct_zscore,pace_rank,pace_zscore,pie_rank,pie_zscore,poss_rank,poss_zscore,offrtg_opp,defrtg_opp,netrtg_opp,ast_pct_opp,ast_v_tov_opp,ast_ratio_opp,oreb_pct_opp,dreb_pct_opp,reb_pct_opp,tov_pct_opp,efg_pct_opp,ts_pct_opp,pace_opp,pie_opp,poss_opp,offrtg_rank_opp,offrtg_zscore_opp,defrtg_rank_opp,defrtg_zscore_opp,netrtg_rank_opp,netrtg_zscore_opp,ast_pct_rank_opp,ast_pct_zscore_opp,ast_v_tov_rank_opp,ast_v_tov_zscore_opp,ast_ratio_rank_opp,ast_ratio_zscore_opp,oreb_pct_rank_opp,oreb_pct_zscore_opp,dreb_pct_rank_opp,dreb_pct_zscore_opp,reb_pct_rank_opp,reb_pct_zscore_opp,tov_pct_rank_opp,tov_pct_zscore_opp,efg_pct_rank_opp,efg_pct_zscore_opp,ts_pct_rank_opp,ts_pct_zscore_opp,...,fga_c3_opp,fg_pct_c3_opp,fgm_atb3_opp,fga_atb3_opp,fg_pct_atb3_opp,fgm_ra_rank_opp,fgm_ra_zscore_opp,fga_ra_rank_opp,fga_ra_zscore_opp,fg_pct_ra_rank_opp,fg_pct_ra_zscore_opp,fgm_paint_rank_opp,fgm_paint_zscore_opp,fga_paint_rank_opp,fga_paint_zscore_opp,fg_pct_paint_rank_opp,fg_pct_paint_zscore_opp,fgm_mr_rank_opp,fgm_mr_zscore_opp,fga_mr_rank_opp,fga_mr_zscore_opp,fg_pct_mr_rank_opp,fg_pct_mr_zscore_opp,fgm_lc3_rank_opp,fgm_lc3_zscore_opp,fga_lc3_rank_opp,fga_lc3_zscore_opp,fg_pct_lc3_rank_opp,fg_pct_lc3_zscore_opp,fgm_rc3_rank_opp,fgm_rc3_zscore_opp,fga_rc3_rank_opp,fga_rc3_zscore_opp,fg_pct_rc3_rank_opp,fg_pct_rc3_zscore_opp,fgm_c3_rank_opp,fgm_c3_zscore_opp,fga_c3_rank_opp,fga_c3_zscore_opp,fg_pct_c3_rank_opp,fg_pct_c3_zscore_opp,fgm_atb3_rank_opp,fgm_atb3_zscore_opp,fga_atb3_rank_opp,fga_atb3_zscore_opp,fg_pct_atb3_rank_opp,fg_pct_atb3_zscore_opp,opp_fgm_ra,opp_fga_ra,opp_fg_pct_ra,opp_fgm_paint,opp_fga_paint,opp_fg_pct_paint,opp_fgm_mr,opp_fga_mr,opp_fg_pct_mr,opp_fgm_lc3,opp_fga_lc3,opp_fg_pct_lc3,opp_fgm_

In [8]:
REGULAR_SEASON_DATES_ADJUSTED = {
    "2013-2014": (datetime.date(2013, 11, 12), datetime.date(2014, 4, 2)),
    "2014-2015": (datetime.date(2014, 11, 11), datetime.date(2015, 4, 1)),
    "2015-2016": (datetime.date(2015, 11, 10), datetime.date(2016, 3, 30)),
    "2016-2017": (datetime.date(2016, 11, 8), datetime.date(2017, 3, 29)),
    "2017-2018": (datetime.date(2017, 10, 31), datetime.date(2018, 3, 28)),
    "2018-2019": (datetime.date(2018, 10, 30), datetime.date(2019, 3, 27)),
    "2019-2020": (datetime.date(2019, 11, 5), datetime.date(2020, 2, 26)),
    "2020-2021": (datetime.date(2021, 1, 5), datetime.date(2021, 5, 2)),
    "2021-2022": (datetime.date(2021, 11, 2), datetime.date(2022, 3, 27)),
    "2022-2023": (datetime.date(2022, 11, 1), datetime.date(2023, 3, 26)),
}

In [9]:
def prepare_data(
    df,
    model_type,
    feature_types=["main"],
    random_state=17,
    test_size=0.2,
    scale_data=True,
    time_based_split=False,
    print_shape=False,
):
    """
    Preprocesses the given data frame for use in a machine learning model.

    Args:
        df (pandas DataFrame): The data to preprocess.
        model_type (str): The type of model to train, either "CLS" for classification or "REG" for regression.
        feature_types (list of str, optional): The types of features to include in the training data. Can be any combination of "main", "rank", "zscore", "other", or "all". Defaults to ["main"].
        random_state (int, optional): The random state to use for splitting the data into train and test sets. Defaults to 17.
        test_size (float, optional): The proportion of the data to use for the test set when splitting. Defaults to 0.2.
        scale_data (bool, optional): Whether to scale the features using the StandardScaler. Defaults to True.
        time_based_split (bool, optional): Whether to split the data into train and test sets based on a time-based split (i.e. training data comes before a certain date and test data comes after that date). Defaults to False.
        print_shape (bool, optional): Whether to print the shapes of the resulting train and test sets. Defaults to False.

    Returns:
        tuple of numpy ndarrays: The resulting train and test sets, in the order (X_train, X_test, y_train, y_test). If `scale_data` is True, the scaler used for feature scaling is also returned as a second item in the tuple.
    """

    # Restrict Dates to Regular Season minus First and Last 2 Weeks
    df["game_date"] = pd.to_datetime(df["game_id"].str[:8], format="%Y%m%d")
    mask = pd.Series(False, index=df.index)
    for season, (start_date, end_date) in REGULAR_SEASON_DATES_ADJUSTED.items():
        season_mask = (df["game_date"] >= pd.to_datetime(start_date)) & (
            df["game_date"] <= pd.to_datetime(end_date)
        )
        mask |= season_mask
    df = df[mask]

    # Determine the target column
    target = None
    if model_type == "CLS":
        target = "CLS_TARGET_home_margin_GT_home_spread"
    elif model_type == "REG":
        target = "REG_TARGET_actual_home_margin"
    else:
        raise ValueError("model_type must be either 'CLS' or 'REG'")

    # Determine Features to Use
    drop_columns = [
        "fd_line_home",
        "dk_line_home",
        "cover_consensus_home",
        "game_id",
        "game_date",
        "REG_TARGET_actual_home_margin",
        "CLS_TARGET_home_margin_GT_home_spread",
    ]

    # FEATURE OPTIONS (main, rank, zscore, other, all)
    main_features = [
        "home_team_num",
        "away_team_num",
        "home_spread",
        "league_year_end",
        "day_of_season",
        "elo1_pre",
        "elo2_pre",
        "elo_prob1",
        "elo_prob2",
    ]
    rank_features = [feature for feature in list(df) if "rank" in feature]
    zscore_features = [feature for feature in list(df) if "zscore" in feature]
    other_features = [
        feature
        for feature in list(df)
        if feature not in [target] + main_features + drop_columns
    ]
    all_features = main_features + other_features

    features_to_use = []
    if "all" in feature_types:
        features_to_use = all_features
    elif "main" in feature_types:
        features_to_use = main_features
        if "other" in feature_types and (
            "rank" in feature_types or "zscore" in feature_types
        ):
            raise ValueError("other_features can only be added to main features.")
        elif "other" in feature_types:
            features_to_use += other_features
        elif "other" not in feature_types:
            if "rank" in feature_types:
                features_to_use += rank_features
            if "zscore" in feature_types:
                features_to_use += zscore_features
    else:
        raise ValueError(
            "Feature_types must be one of 'all', 'main', 'rank', 'zscore', 'other'."
        )
    df = df[[target] + features_to_use]

    # Convert Data Types
    float_cols = [
        col
        for col in df.columns
        if col
        not in [
            "game_id",
            "CLS_TARGET_home_margin_GT_home_spread",
            "REG_TARGET_actual_home_margin",
        ]
    ]
    df[float_cols] = df[float_cols].astype("float32")

    # Drop NA Values
    df = df.dropna()

    # Split Data
    X_train, X_test, y_train, y_test = None, None, None, None
    if time_based_split:
        test_data = df[df["league_year_end"] == 22]
        train_data = df[df["league_year_end"] < 22]
        X_train = train_data.drop(target, axis=1)
        X_test = test_data.drop(target, axis=1)
        y_train = train_data[target]
        y_test = test_data[target]
    else:
        X_train, X_test, y_train, y_test = train_test_split(
            df.drop(target, axis=1),
            df[target],
            test_size=test_size,
            random_state=random_state,
        )
    
    # Feature Scaling
    scaler = None
    if scale_data:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

    if print_shape:
        print(f"X_train shape: {X_train.shape}")
        print(f"X_test shape: {X_test.shape}")
        print(f"y_train shape: {y_train.shape}")
        print(f"y_test shape: {y_test.shape}")
    return (X_train, X_test, y_train, y_test), scaler

## AutoKeras - Regression

* https://autokeras.com/tutorial/structured_data_regression/

### Setup and Preprocessing

In [ ]:
(X_train, X_test, y_train, y_test), scalar = prepare_data(df, 'REG', feature_types=["main", "rank", "zscore"],
                                                random_state=17, test_size=0.2,
                                                scale_data=False, time_based_split=False,
                                                print_shape=True)

### Fit

In [ ]:
reg = ak.StructuredDataRegressor(max_trials=20, overwrite=True)

In [ ]:
reg.fit(X_train, y_train)

### Predict

In [ ]:
y_pred = reg.predict(X_test)

### Evaluate

In [ ]:
print(reg.evaluate(X_test, y_test))

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
r2_score_ = r2_score(y_test, y_pred)
print("\nMean Absolute Error: %.2f" % mae)
print("R2_Score: %.2f" % r2_score_)

### Model Finalization and Storage

In [ ]:
final_reg_model = reg.export_model()

In [ ]:
final_reg_model.summary()

In [ ]:
print(type(final_reg_model))

In [ ]:
# try:
#     final_reg_model.save('../models/AutoKeras/AutoKeras_REG_Main_Rank_ZScore', save_format="tf")
# except Exception:
#     final_reg_model.save('../models/AutoKeras/AutoKeras_REG_Main_Rank_ZScore.h5')

## Classification

### Setup and Preprocessing

In [10]:
(X_train, X_test, y_train, y_test), scalar = prepare_data(df, 'CLS', feature_types=["main", "rank", "zscore"],
                                                random_state=17, test_size=0.2,
                                                scale_data=False, time_based_split=False,
                                                print_shape=True)

X_train shape: (4217, 613)
X_test shape: (1055, 613)
y_train shape: (4217,)
y_test shape: (1055,)


### Fit

In [11]:
cls = ak.StructuredDataClassifier(max_trials=20, overwrite=True)

In [12]:
cls.fit(X_train, y_train)

Trial 20 Complete [00h 01m 30s]
val_accuracy: 0.524848461151123

Best val_accuracy So Far: 0.5563636422157288
Total elapsed time: 00h 30m 34s
INFO:tensorflow:Oracle triggered exit
Epoch 1/9
132/132 [==============================] - 6s 9ms/step - loss: 0.7388 - accuracy: 0.4921
Epoch 2/9
132/132 [==============================] - 1s 10ms/step - loss: 0.7025 - accuracy: 0.5231
Epoch 3/9
132/132 [==============================] - 1s 10ms/step - loss: 0.7013 - accuracy: 0.5241
Epoch 4/9
132/132 [==============================] - 1s 10ms/step - loss: 0.6868 - accuracy: 0.5490
Epoch 5/9
132/132 [==============================] - 1s 10ms/step - loss: 0.6844 - accuracy: 0.5459
Epoch 6/9
132/132 [==============================] - 1s 10ms/step - loss: 0.6720 - accuracy: 0.5822
Epoch 7/9
132/132 [==============================] - 1s 10ms/step - loss: 0.6616 - accuracy: 0.6016
Epoch 8/9
132/132 [==============================] - 1s 10ms/step - loss: 0.6474 - accuracy: 0.6230
Epoch 9/9
132/132 [==

INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


### Predict

In [13]:
y_pred = cls.predict(X_test)

33/33 [==============================] - 0s 10ms/step


### Evaluate

In [14]:
print(cls.evaluate(X_test, y_test))
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
print("\nAccuracy: %.3f" % accuracy)
print("Precision: %.3f" % precision)

33/33 [==============================] - 3s 10ms/step - loss: 0.7284 - accuracy: 0.5100
[0.7283919453620911, 0.5099526047706604]

Accuracy: 0.510
Precision: 0.483


### Model Finalization and Storage

In [15]:
final_cls_model = cls.export_model()

In [16]:
final_cls_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 613)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 613)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 613)              1227      
 n)                                                              
                                                                 
 dense (Dense)               (None, 16)                9824      
                                                                 
 re_lu (ReLU)                (None, 16)                0         
                                                                 
 dropout (Dropout)           (None, 16)                0     

In [17]:
print(type(final_cls_model))

<class 'keras.engine.functional.Functional'>


In [18]:
try:
    final_cls_model.save('../models/AutoKeras/AutoKeras_CLS_Main_Rank_ZScore', save_format="tf")
except Exception:
    final_cls_model.save('../models/AutoKeras/AutoKeras_CLS_Main_Rank_ZScore.h5')

INFO:tensorflow:Assets written to: ../models/AutoKeras/AutoKeras_CLS_Main_Rank_ZScore/assets


INFO:tensorflow:Assets written to: ../models/AutoKeras/AutoKeras_CLS_Main_Rank_ZScore/assets
